In [ ]:
import json
import os

# Define the paths to the directories
genre_dir = '/content/drive/MyDrive/genre/U Got Me'
mood_dir = '/content/drive/MyDrive/moods/U Got Me'
key_dir = '/content/drive/MyDrive/tempo_key/U Got Me'

# Get the filenames in the genre, mood, and key directories
genre_files = [f for f in os.listdir(genre_dir) if f.endswith('_genre_predictions.json')]
mood_files = [f for f in os.listdir(mood_dir) if f.endswith('_mood_metadata.json')]
key_files = [f for f in os.listdir(key_dir) if f.endswith('.json')]

# Extract base names from files
def extract_base_name(filename, suffix):
    return filename.replace(suffix, '')

# Create dictionaries to store the base names and their corresponding files
genre_base_names = {extract_base_name(f, '_genre_predictions.json'): f for f in genre_files}
mood_base_names = {extract_base_name(f, '_mood_metadata.json'): f for f in mood_files}
key_base_names = {extract_base_name(f, '.json'): f for f in key_files}

# Print the extracted base names from each directory
print("Genre base names:", genre_base_names.keys())
print("Mood base names:", mood_base_names.keys())
print("Key base names:", key_base_names.keys())

# Find common base names
common_base_names = set(genre_base_names.keys()) & set(mood_base_names.keys()) & set(key_base_names.keys())

# Ensure that there are matching files
if not common_base_names:
    raise ValueError("No matching base names found across the directories!")

# Directory to save combined metadata
output_dir = '/content/U Got Me'
os.makedirs(output_dir, exist_ok=True)

# Process each matching set of files
for base_name in common_base_names:
    # Define file paths
    genre_file_path = os.path.join(genre_dir, genre_base_names[base_name])
    mood_file_path = os.path.join(mood_dir, mood_base_names[base_name])
    key_file_path = os.path.join(key_dir, key_base_names[base_name])

    # Load the data
    with open(genre_file_path, 'r') as genre_file:
        genre_data = json.load(genre_file)

    with open(mood_file_path, 'r') as mood_file:
        mood_data = json.load(mood_file)

    with open(key_file_path, 'r') as key_file:
        key_data = json.load(key_file)

    # Sort genre data by confidence
    genre_data_sorted = sorted(genre_data, key=lambda x: float(x['confidence'].replace('%', '')), reverse=True)

    # Sort mood data by score
    mood_data_sorted = sorted(mood_data, key=lambda x: x['highest_score_prediction']['score'], reverse=True)

    # Sort key data by strength
    key_data_sorted = sorted(key_data, key=lambda x: x['strength'], reverse=True)

    # Extract top 3 unique genres
    genres = []
    seen_genres = set()
    for entry in genre_data_sorted:
        if entry['genre'] not in seen_genres:
            genres.append(entry['genre'])
            seen_genres.add(entry['genre'])
        if len(genres) == 3:
            break

    # Extract top 3 unique moods
    moods = []
    seen_moods = set()
    for entry in mood_data_sorted:
        if entry['highest_score_prediction']['label'] not in seen_moods:
            moods.append(entry['highest_score_prediction']['label'])
            seen_moods.add(entry['highest_score_prediction']['label'])
        if len(moods) == 3:
            break

    # Extract top 3 unique keys
    keys = []
    seen_keys = set()
    for entry in key_data_sorted:
        if entry['key'] not in seen_keys:
            keys.append(entry['key'])
            seen_keys.add(entry['key'])
        if len(keys) == 3:
            break

    # Calculate average BPM
    total_bpm = sum(entry['tempo_bpm'] for entry in key_data)
    average_bpm = total_bpm / len(key_data) if key_data else 0

    # Add song name
    song_name = base_name.split('_')[0].replace(' ', '_')  # Replace spaces with underscores

    # Combine the data
    output_data = [{
        "song_name": song_name,
        "genres": genres,
        "moods": moods,
        "keys": keys,
        "bpm": average_bpm
    }]

    # Define the output file path
    output_file_path = os.path.join(output_dir, f'{base_name}_combined_metadata.json')

    # Save the combined data to a JSON file
    with open(output_file_path, 'w') as output_file:
        json.dump(output_data, output_file, indent=4)

    print(f"Combined metadata saved to '{output_file_path}'")


Genre base names: dict_keys(['U Got Me'])
Mood base names: dict_keys(['U Got Me'])
Key base names: dict_keys(['U Got Me'])
Combined metadata saved to '/content/U Got Me/U Got Me_combined_metadata.json'


In [ ]:
import shutil
import os

# Use the output directory from the first block
output_dir = '/content/Scrufizzer)'  # This is the directory where your combined metadata files are stored

# Define the destination directory on Google Drive, including the directory name you want to create
destination_dir = '/content/drive/MyDrive/Processed_Metadata/Scrufizzer)'  # This will create a 'Wild' folder in your Google Drive

# Ensure the destination directory exists
os.makedirs(os.path.dirname(destination_dir), exist_ok=True)

# Copy the entire directory, including the directory itself
shutil.copytree(output_dir, destination_dir, dirs_exist_ok=True)

print(f"Output directory '{output_dir}' copied to '{destination_dir}' on Google Drive.")


Output directory '/content/Scrufizzer)' copied to '/content/drive/MyDrive/Processed_Metadata/Scrufizzer)' on Google Drive.
